In [1]:
import ROOT
import numpy as np
import pylab as plt
from array import array
import pandas as pd
from sqlalchemy import *
import model
from sqlalchemy import orm
%matplotlib inline

Welcome to JupyROOT 6.08/00


In [2]:
ROOT.gROOT.ProcessLine(
"struct tbjcStruct\
{\
    short uval[512];\
};" )

engineMap = create_engine('sqlite+pysqlite:///../data/SQL/Tin132/map.sql')
LookMapdf = pd.io.sql.read_sql("SELECT * FROM LookMap", engineMap)

def FindPadNum(AgetID,ChannelID):
    return LookMapdf[(LookMapdf['AgetID']==AgetID)&(LookMapdf['ChannelID']==ChannelID)]['PadNum'].values[0]

In [3]:
f = ROOT.TFile("../data/ANL_March_2013/ROOT_files/run40.root")
tree = f.Get("AutoTree")

engine = create_engine('sqlite+pysqlite:///../b.sql')

model.metadata.bind = engine
model.metadata.create_all()

sm = orm.sessionmaker(bind=engine, autoflush=True, autocommit=False, expire_on_commit=True)
session = orm.scoped_session(sm)


In [ ]:
for AgetID in range(4):
    print "alalal"
    for ChannelID in range(68):

        continue
        tb0 = tree.GetBranch("asic_{}_{:02d}".format(AgetID,ChannelID))
        tbjcStruct = ROOT.tbjcStruct()
        tb0.SetAddress(ROOT.AddressOf(tbjcStruct,'uval'))
        
        PadNum = FindPadNum(AgetID,ChannelID)
        
        TB = 512
        for i in xrange(tb0.GetEntries()):

            if i % 100 == 0:
                print i

            continue
            tb0.GetEntry(i)
            RawADC = model.RawADC()
            RawADC.EventID = i
            RawADC.PadNum = PadNum

            for j in range(TB):
                setattr(RawADC, 't'+str(j), tbjcStruct.uval[j])
            session.add(RawADC)

            #session.flush()
            if i %10000 == 0:
                session.commit()
        #session.commit()
        